<h2>Construct keyfiles from project directory containing a Base FVS Rx template.</h2>

In [ ]:
import os
import glob
from jinja2 import Template

Identify stands to run.

In [ ]:
# stands = ['25123'] # all stands to create keyfiles

Create a jinja2 template from a Base_Rx.key file.

In [ ]:
# read in the base_rx keyfile template using jinja2 templating
with open(os.path.join('Rx_Template','Base_Rx.key'), 'r') as base_keyfile:
    template = Template(base_keyfile.read())
    print('Found Base_Rx.key and created jinja2 template.')

A dictionary for holding the items to insert into an FVS keyfile template using jinja2 templating.

In [ ]:
inserts = {}

Specify the FVS input and output databases for insertion in the jinja2 template

In [ ]:
inserts['FVSIn'] = 'FVSIn'
inserts['FVSOut'] = 'FVSOut'

Look in the Rx_Template directory, add all the kcp files to the `inserts` dictionary.

In [ ]:
template_kcps = glob.glob(os.path.join('Rx_Template','*.kcp'))
if len(template_kcps) > 0:
    print('Found the following kcp files in the Rx_Template directory:')
    for kcp in template_kcps:
        fname = os.path.split(kcp)[-1]
        print(fname, end='...')
        # read the kcp file
        key = fname.split('.')[0] # key for item in inserts dictionary
        with open(kcp, 'r') as item:
            value = item.read()
        # add the contents of the kcp file to the inserts dictionary
        inserts[key] = value
        print(' added to template.')
else: 
    raise FileNotFoundError('No kcp files found in the Rx_Template directory.')

Read the contents of each rx*.kcp file in the Rxs directory and store them as values in an `rxs_dict` dictionary.

In [ ]:
rxs_dict = {}
# a dictionary storing the silvicultural keywords for each rx
rx_kcps = glob.glob(os.path.join('Rx_Template', 'Rxs', 'rx*.kcp'))
if len(rx_kcps) > 0:
    print('Found the following kcp files in the Rxs subdirectory:')
    for kcp in rx_kcps:
        fname = os.path.split(kcp)[-1]
        print(fname, end='...')
        # read the kcp file
        key = fname.split('/')[-1].split('.')[0] # key for item in inserts dictionary
        with open(kcp, 'r') as item:
            value = item.read()
        # add the contents of the kcp file to the inserts dictionary
        rxs_dict[key] = value
        print(' added to template.')
else: 
    raise FileNotFoundError('No kcp files found in the Rx_Template directory.')

A function to use for creating keyfiles.

In [ ]:
def create_keyfile(standID, rx, offset):
    '''
    Creates a single FVS keyfile based on the jinja2 template.
    '''
    inserts['ID'] = standID
    inserts['rx'] = rxs_dict[rx]
    inserts['offset'] = offset
    with open(os.path.join('keyfiles_to_run','stand'+str(standID)+'_'+rx+'_off'+str(offset)+'.key'),'w') as keyfile:
        keyfile.write(template.render(**inserts))

In [ ]:
def create_keyfiles(stands, rxs, offsets=[0], verbose=True):
    '''
    Creates FVS keyfiles for all stands using Base_Rx.key as a template.
    Arguments:
    stands: List of standIDs that keyfiles will be created for. Required.
    rxs: a list of rx names to build keyfiles for. Required.
    offsets: optional, a list of offsets, used in FVS to delay implementation of a 
        management regime. e.g., [0, 5, 10]. Defaults to a list with no offsets (i.e., [0]).
    '''
    stands_processed = 0
    keyfiles_written = 0
    num_stands = len(stands)
    num_keys = num_stands * len(rxs) * len(offsets)
    print('Creating', num_keys, 'keyfiles for', num_stands, 'stands.')

    for standID in stands:
        if verbose:
            print('Creating keyfiles for stand', standID, end='... ')
        stand_keyfiles = 0
        for rx in rxs:
            for offset in offsets:
                create_keyfile(standID, rx, offset) # run the create_keyfile function
                keyfiles_written += 1
                stand_keyfiles += 1
        stands_processed += 1
        if verbose:
            print(stand_keyfiles, 'keyfiles written.')
        else:
            if stands_processed % 100 == 0:
                print(stands_processed, end='... ')
    print('Done. Created', keyfiles_written, 'keyfiles for', stands_processed, 'stands.')

Create the keyfiles!

In [ ]:
# stands = [...]

In [ ]:
%%time
create_keyfiles(stands=stands, rxs=['rx1', 'rx2', 'rx3', 'rx4', 'rx5'], verbose=False)